# Mesh Segmentation using Heat Kernel Signature


In [ ]:
# ============================================================
# Setup for Mesh Segmentation using Heat Kernel Signature (HKS)
# This cell initialises the necessary libraries.
# ============================================================
__author__ = "Faisal Jayousi"

from pathlib import Path  # For handling file paths

import matplotlib.pyplot as plt  # For plotting visualizations
import meshplot as mp  # For 3D mesh visualization
import numpy as np
import sklearn.preprocessing as skp

from geotopomethods import SegmentMesh

## Utility functions


In [ ]:
def off2numpy(shape_name):
    """
    Converts a .off file to NumPy arrays representing vertices and faces.

    Parameters:
        shape_name (str): Path to the .off file.

    Returns:
        tuple: (vertices, faces) where:
            - vertices (np.ndarray): Array of vertex coordinates.
            - faces (np.ndarray): Array of face indices.
    """
    with open(shape_name, "r") as file:
        # Skip the first line and read number of vertices and faces
        file.readline()
        num_vertices, num_faces, _ = map(int, file.readline().split())

        # Read the remaining lines
        data = file.readlines()

    # Parse vertices and faces
    vertices = np.array(
        [list(map(float, line.split())) for line in data[:num_vertices]]
    )
    faces = np.array(
        [list(map(int, line.split()[1:])) for line in data[num_vertices:]]
    )

    return vertices, faces


def get_labels(label_name, num_faces):
    """
    Reads label information and associates labels with mesh faces.

    Parameters:
        label_name (str): Path to the label file.
        num_faces (int): Number of faces in the mesh.

    Returns:
        np.ndarray: Array of labels for each face.
    """
    labels_array = np.empty([num_faces], dtype="|S100")

    with open(label_name, "r") as S:
        info = S.readlines()

    labels, face_indices = info[0::2], info[1::2]

    for i, lab in enumerate(labels):
        indices = [int(f) - 1 for f in face_indices[i].split(" ")[:-1]]
        labels_array[np.array(indices)] = lab[:-1]
    return labels_array

## Loading and Visualising Mesh Data


In [ ]:
dataset_path = ".." / Path('data')
vertices, faces = off2numpy(dataset_path / 'Human' / '20.off')
label_faces = get_labels(dataset_path / 'Human' / '20_labels.txt', len(faces))

mp.plot(vertices, faces, c=skp.LabelEncoder().fit_transform(label_faces))

## Mesh Segmentation


In [ ]:
def hks(t, eigenvalues: np.ndarray, eigenvectors: np.ndarray):
    """
    Compute Heat Kernal Signature (HKS) for a given t.

    Args:
        t : float
            Time scale parameter.
        eigenvalues : np.ndarray
            1D array of eigenvalues obtained from the Laplacian
        eigenvectors : np.ndarray
            2D array of eigenvectors. Shape: (n_points, n_eigenvalues)
    """
    exp_decay = np.exp(-eigenvalues * t)  # Shape (n_eigenvalues,)
    hks_values = np.sum(exp_decay[None, :] * np.square(eigenvectors), axis=1)
    return hks_values

In [ ]:
mhs = SegmentMesh(vertices, faces)
mhs.compute_laplacian(k=200)
mhs.build_neighbourhood_graph()

weights = hks(0.1, mhs.eigenvalues, mhs.eigenvectors)
tomato = mhs(weights=weights)

tomato.plot_diagram()
plt.show()

In [ ]:
tomato.n_clusters_ = 5
mp.plot(vertices, faces, c=tomato.labels_)